# Crawl Data from Top Youtuber in Taiwan

##  1. Get Famous Youtuber Link

In [2]:
'''
###########################################################
    Import Libraries
###########################################################
'''
import numpy as np
import requests
import sys
import math
import time
from bs4 import BeautifulSoup
import functools
import csv
import pandas as pd

### Get the top 250 youtubers link and basic attributes in taiwan. Sorted by subscribers num

In [2]:
'''
######################################################################################################
    Get the top 250 youtubers link in socialblade website
######################################################################################################
'''

top_youtuber_rank_url = "https://socialblade.com/youtube/top/country/tw/mostsubscribed"

content = requests.get(top_youtuber_rank_url).text
page_soup = BeautifulSoup(content, "html.parser")

# Find potential user position 
div_class = page_soup.find_all('a', href=True)

youtuber_socialblade = [] # Name, url in socialblade
for num in range(0, len(div_class)):
    href = div_class[num]
    url = href.get('href')
    if url.find("/youtube/user/") is not -1:
        youtuber_socialblade.append([href.text, "https://socialblade.com"+url])

print("Top 250 Youtuber: ")
for i in range(0, len(youtuber_socialblade)):
    print(i+1, 'is', youtuber_socialblade[i][0], youtuber_socialblade[i][1])

Top 250 Youtuber: 
1 is 這群人TGOP https://socialblade.com/youtube/user/e84768131
2 is 滾石唱片 ROCK RECORDS https://socialblade.com/youtube/user/rockrecordstaipei
3 is 相信音樂BinMusic https://socialblade.com/youtube/user/binmusictaipei
4 is 阿滴英文 https://socialblade.com/youtube/user/eo3jwe3hezuwfdvcehqk9q
5 is 蔡阿嘎 https://socialblade.com/youtube/user/kyoko38
6 is 阿神 https://socialblade.com/youtube/user/charlie615119
7 is 杰威爾音樂 JVR Music https://socialblade.com/youtube/user/jvrmuzic
8 is 華納音樂 Warner Music Taiwan https://socialblade.com/youtube/user/warnertaiwan
9 is 聖結石Saint https://socialblade.com/youtube/user/idhd_1spj49unbwx1fjs2a
10 is 眾量級CROWD https://socialblade.com/youtube/user/3lbfxbwtebdooub8-qjm9q
11 is 瑪莎與熊 Masha and The Bear CH https://socialblade.com/youtube/user/fsxflbi8fpnavq_rl3i4bw
12 is 福茂唱片 https://socialblade.com/youtube/user/linfairrecords
13 is 華研國際 https://socialblade.com/youtube/user/himservice
14 is 放火 Louis https://socialblade.com/youtube/user/louislee0602
15 is 小玉 https

In [3]:
'''
######################################################################################################
    Get youtuber name, url, UPLOADS, SUBSCRIBERS, VIDEO VIEWS, Country, CHANNEL TYPE, USER CREATED DATE
######################################################################################################
'''
youtuber = []
for i in range(0, len(youtuber_socialblade)): 
    user = youtuber_socialblade[i]
    url = user[1]
    content = requests.get(url).text
    page_soup = BeautifulSoup(content, "html.parser")
    attributes = []
    attributes.append(user[0])
    
    # Get youtube url
    youtube_link_class = page_soup.find_all("a", {"class": "core-button -margin core-small-wide ui-black"})
    for link_text in youtube_link_class:
        link_url = link_text.get('href')
        if link_url is not None:
            attributes.append(link_url)
    
    # Get attributes value
    info_class = page_soup.find_all("div", {"class": "YouTubeUserTopInfo"})
    for info in info_class:
        value = info.find_all("span", {"style": "font-weight: bold;"})[0].text
        attributes.append(value)
    youtuber.append(attributes)
    time.sleep(0.05)
print("Done")

Done


In [5]:
print(type(youtuber))
for i in range(0, len(youtuber)):
    for j in youtuber[i]:
        print(j, end=" "),
    print()

<class 'list'>
這群人TGOP https://youtube.com/channel/UC6FcYHEm7SO1jpu5TKjNXEA 177 2,796,566 598,312,864 TW Entertainment Jun 7th, 2008 
滾石唱片 ROCK RECORDS https://youtube.com/channel/UCPcF3KTqhD67ADkukx_OeDg 4,276 2,233,642 2,550,899,226 TW Music Jun 8th, 2010 
相信音樂BinMusic https://youtube.com/channel/UC7ovu6a8ydIbDy0fAKmoZ9A 1,208 2,121,542 2,303,425,117 TW Music Aug 10th, 2009 
阿滴英文 https://youtube.com/channel/UCeo3JwE3HezUWFdVcehQk9Q 353 1,964,359 144,282,402 TW Education Jan 11th, 2015 
蔡阿嘎 https://youtube.com/channel/UCPwxSX0DYDMQxCvgfeVDv_g 548 1,936,141 422,444,360 TW Entertainment Jun 3rd, 2006 
阿神 https://youtube.com/channel/UCnJEWsS5agXCkqIpyHC9Grg 3,524 1,731,629 656,374,339 TW Games May 11th, 2009 
杰威爾音樂 JVR Music https://youtube.com/channel/UCKUlsqazP-4QmxdEtUPlxOA 516 1,450,785 2,030,594,707 TW Music Sep 2nd, 2010 
華納音樂 Warner Music Taiwan https://youtube.com/channel/UCStpPtoeQUYII1Jsbl0qf8A 1,700 1,419,171 907,535,393 TW Music Jul 6th, 2009 
聖結石Saint https://youtube.com/cha

In [12]:
'''
######################################################################################################
    Preprocess data about comma and date
######################################################################################################
'''
month_dictionary = {'Jan':'01', 'Feb':'02', "Mar":'03', "Apr":'04', "May":'05', "Jun":'06', 
                    "Jul":'07', "Aug":'08', "Sep":'09', "Oct":'10', "Nov":'11', "Dec":'12'}
date_dictionary = {'1':'01', '2':'02', '3':'03', '4':'04', '5':'05', '6':'06', '7':'07', '8':'08', '9':'09'}

def remove_comma(value_string):
    if type(value_string) == int:
        return value_string
    value = 0
    for i in range(0, len(value_string)):
        if(value_string[i]!=","):
            value = value*10 + int(value_string[i])
    return int(value)
       
def date_to_normal_express(value_string):
    if type(value_string) == int or len(value_string) == 10:
        return value_string
    value_string = value_string.replace(',', " ")
    value_string = value_string.replace('th', "")
    value_string = value_string.replace('rd', "")
    value_string = value_string.replace('nd', "")
    value_string = value_string.replace('st', "")
    
    date = value_string.split()
    date[0] = month_dictionary[date[0]]
    if(len(date[1]) == 1):
        date[1] = date_dictionary[date[1]]
    
    date = date[2] + "-" + date[0] + "-" + date[1]
    
    return (date)

today = "2018-12-25"

for i in range(0, len(youtuber)):
    youtuber[i][0] = youtuber[i][0].replace(';', "")
    youtuber[i][2] = remove_comma(youtuber[i][2]) #UPLOADS
    youtuber[i][3] = remove_comma(youtuber[i][3]) #SUBSCRIBERS
    youtuber[i][4] = remove_comma(youtuber[i][4]) #VIDEO VIEWS
    youtuber[i][7] = date_to_normal_express(youtuber[i][7]) #Date from string to int
    youtuber[i].append(today)
    
    # calculate the channel founded dates
    start_date = youtuber[i][7]
    end_data = today
    start_sec = time.mktime(time.strptime(start_date,'%Y-%m-%d'))
    end_sec = time.mktime(time.strptime(end_date,'%Y-%m-%d'))
    work_days = int((end_sec - start_sec)/(24*60*60))
    youtuber[i].append(work_days)

    
for i in range(0, len(youtuber)):
    for j in youtuber[i]:
        print(j, end=" "),
    print()

這群人TGOP https://youtube.com/channel/UC6FcYHEm7SO1jpu5TKjNXEA 177 2796566 598312864 TW Entertainment 2008-06-07 2018-12-25 3853 
滾石唱片 ROCK RECORDS https://youtube.com/channel/UCPcF3KTqhD67ADkukx_OeDg 4276 2233642 2550899226 TW Music 2010-06-08 2018-12-25 3122 
相信音樂BinMusic https://youtube.com/channel/UC7ovu6a8ydIbDy0fAKmoZ9A 1208 2121542 2303425117 TW Music 2009-08-10 2018-12-25 3424 
阿滴英文 https://youtube.com/channel/UCeo3JwE3HezUWFdVcehQk9Q 353 1964359 144282402 TW Education 2015-01-11 2018-12-25 1444 
蔡阿嘎 https://youtube.com/channel/UCPwxSX0DYDMQxCvgfeVDv_g 548 1936141 422444360 TW Entertainment 2006-06-03 2018-12-25 4588 
阿神 https://youtube.com/channel/UCnJEWsS5agXCkqIpyHC9Grg 3524 1731629 656374339 TW Games 2009-05-11 2018-12-25 3515 
杰威爾音樂 JVR Music https://youtube.com/channel/UCKUlsqazP-4QmxdEtUPlxOA 516 1450785 2030594707 TW Music 2010-09-02 2018-12-25 3036 
華納音樂 Warner Music Taiwan https://youtube.com/channel/UCStpPtoeQUYII1Jsbl0qf8A 1700 1419171 907535393 TW Music 2009-07-06 20

In [11]:
print(len(youtuber[0][7]))

10


## 2. Write basic information of youtuber

In [22]:
'''
######################################################################################################
    Write the basic information of youtuber to csv file
######################################################################################################
'''

with open('youtuber.csv', 'w+', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['User_Name', 'Link', 'Uploads', 'Subscribers', 'Total_Video_Views', 'Country', \
                        'Channel_Type', 'User_Created_Date', "Today_Date", 'User_Exist_Days'])
    for i in range(len(youtuber)):
        decode = []
        for info in youtuber[i]:
            if type(info) == str:
                decode.append(info.encode(sys.stdin.encoding, "replace").decode(sys.stdin.encoding))
            else:
                decode.append(info)
        #print(decode)
        writer.writerow(decode)
print("Done")

Done
